In [17]:
from src.indexing_utils import load_docs, create_docstore
from llama_index.embeddings.gemini import GeminiEmbedding

In [15]:
nodes = load_docs('../../../storage/docstore/summary_nodes', return_docstore=False)

In [16]:
nodes[0].metadata

{'tag': 'p',
 'file_path': '/home/dai/rag/data/clean_html/Articles/Chikitsaa/Aamadosha.htm',
 'file_name': 'Aamadosha.htm',
 'file_type': 'text/html',
 'file_size': 2418,
 'creation_date': '2024-02-15',
 'last_modified_date': '2024-02-14',
 'last_accessed_date': '2024-02-15',
 'summary': 'Aama is undigested food that, when vitiated by doshas (bodily humors), becomes the root cause of many diseases. Treatment involves improving digestion (paachana), stimulating digestion (deepana), and lubrication (snehana). Sweating (svedana) helps move the aama towards the digestive tract, which is then eliminated through the appropriate route (oral, nasal, or anal) using shodhana (purification). However, shodhana should be avoided when aama is scattered throughout the body, attached to tissues, or immobile. It should also not be used when aama is spontaneously moving upwards or downwards. A balanced diet that is easily digestible and does not cause discomfort is recommended.',
 'is_summary_swapped': 

In [18]:
# Removing the is_summary_swapped from the metadata and then storing it again
for node in nodes:
    node.metadata.pop('is_summary_swapped')

create_docstore(nodes=nodes, save_dir="../../../storage/docstore", store_name='summary_nodes_modified')

In [13]:
embed_model = GeminiEmbedding(model_name='models/embedding-001')

# Creating the metadata summary in embeddings
for node in nodes:
    node.embedding = embed_model.get_text_embedding(node.metadata['summary'])

KeyboardInterrupt: 

In [16]:
# from llama_index.core import Settings


In [18]:
len(embed_model.get_text_embedding("What are you doing?"))

768

### Embeddings on summary nodes

In [28]:
from src.indexing_utils import load_docs
import faiss
import numpy as np

In [27]:
nodes = load_docs(path="../../../storage/docstore/summary_node_with_embeddings", return_docstore=False)

In [30]:
# nodes[0].embedding

In [31]:
def create_index(nodes):
    """
    Creates the indexes of the index of documents and return it. 
    """
    # Creating the vector indexes using the index method of Inner Product(Cosine Similarity)
    index = faiss.IndexFlatIP(768)

    for node in nodes:
        index.add(np.array([node.embedding]))

    return index


def search_indexes(model:str, index, query:str, top_similar:int):
    """
    Search the indexes and retrieves the top k similar indexes.
    """
    # Get embeddings of query
    embed_model = GeminiEmbedding()
    query_embedding = embed_model.get_text_embedding(query)
    
    distances, ann = index.search(np.array([query_embedding]), k=top_similar)
    distances = distances.flatten().tolist()
    ann = ann.flatten().tolist()

    return distances, ann


index = create_index(nodes)
query = "How does ayurveda deal with illnesses like HIV AIDS?"
distances, ann = search_indexes(model='models/embedding-001', index=index, query=query, top_similar=2)

In [32]:
distances, ann

([0.7591007351875305, 0.7536753416061401], [296, 416])

In [42]:
print(nodes[296].metadata['summary'])

**Summary of Ayurveda and HIV/AIDS**

Ayurveda, an ancient Indian medical system, recognizes symptoms of HIV/AIDS, despite not explicitly naming the disease. These include fever, weight loss, diarrhea, wasting, herpes, lymphadenopathy, oral candidiasis, aphthous ulcers, Bell's palsy, and tubercular ascites.

Ayurveda attributes HIV/AIDS to immunodeficiency and classifies it as "Dhaatugata Jvara," a condition where the immune system is weakened. Immunity (Vyaadhikshamatva) is categorized into congenital, seasonal, and therapeutic. Therapeutic immunity is further divided into diet-based, exercise-based, and Rasaayana-based.

Rasaayana, a rejuvenating therapy, is considered crucial for strengthening immunity. It includes both medicinal (Dravya) and behavioral (Aachaara) components. Counseling, a key aspect of HIV/AIDS management, aligns with Aachaara Rasaayana.

For asymptomatic HIV/AIDS patients, Ayurveda recommends a tissue-building approach (Santarpan`a Chikitsaa) to restore the immune

In [45]:
print(nodes[416].metadata['summary'])

Ayurvedic medicine, a traditional Indian system of medicine, emphasizes the balance of three doshas: Vata, Pitta, and Kapha. These doshas are believed to govern various functions in the body, and imbalances can lead to disease. Ayurveda also places great importance on diet and lifestyle, with specific recommendations for different body types and conditions.
